In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100

In [10]:
# Load the datasets and investigate 
df1 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/Movie Bechdel Test Scores/Bechdel.csv')
df2 = pd.read_csv('/Users/justynakurach/Documents/GitHub/IronHackLabs/Week 5/FINAL DATASETS SELECTION/Complete IMDB Movies /comleteIMDBmovies.csv')

In [12]:
df1 = df1.drop('Unnamed: 0', axis=1)

In [117]:
df2.fillna(0, inplace=True)


In [118]:
df2['year'] = df2['year'].astype('int64')


In [126]:
df1 = df1.rename(columns={'title': 'movie_title', 'year': 'movie_year'})


In [110]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62058 entries, 0 to 62057
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         62058 non-null  object 
 1   rating        41391 non-null  object 
 2   year          2223 non-null   float64
 3   users_rating  62056 non-null  float64
 4   votes         62056 non-null  object 
 5   metascore     9611 non-null   float64
 6   img_url       58913 non-null  object 
 7   countries     62058 non-null  object 
 8   languages     62058 non-null  object 
 9   actors        62058 non-null  object 
 10  genre         62058 non-null  object 
 11  tagline       38302 non-null  object 
 12  description   58416 non-null  object 
 13  directors     61622 non-null  object 
 14  runtime       49893 non-null  object 
 15  imdb_url      62058 non-null  object 
dtypes: float64(3), object(13)
memory usage: 7.6+ MB


In [101]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9373 entries, 0 to 9372
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   9373 non-null   object 
 1   imdbid  9373 non-null   float64
 2   id      9373 non-null   int64  
 3   rating  9373 non-null   int64  
 4   year    9373 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 366.3+ KB


In [14]:
import pandas as pd
from fuzzywuzzy import fuzz
# Create an empty list to store the potential matches
potential_matches = []

# Iterate over the movie titles in both datasets
for title1 in df1['title']:
    for title2 in df2['title']:
        score = fuzz.ratio(title1, title2)
        if score > 80:
            potential_matches.append((title1, title2, score))

# Create a DataFrame of potential matches
df_matches = pd.DataFrame(potential_matches, columns=['title1', 'title2', 'score'])


In [130]:
df_matches

,title_df1,title_df2,year
0,Tokyo Story,Toy Story 4,1953
1,Children of the Corn,Children of the Corn,1984
2,Friday the 13th: A New Beginning,Friday the 13th: A New Beginning,1985
3,Under the Same Moon,Underneath the Same Moon,2007
4,Undertow,Undertow,2009
5,"Ghost Writer, The",Ghost Writer,2010
6,Kill Your Darlings,Kill Your Darlings,2013
7,Neighbors,Neighbor,2014
8,Spectre,InSpectres,2015


In [129]:
# Create an empty list to store the potential matches
potential_matches = []

# Iterate over the movie titles and years in both datasets
for title1, year1 in zip(df1['movie_title'], df1['movie_year']):
    for title2, year2 in zip(df2['title'], df2['year']):
        # Check if the year columns match and the title score is greater than 80
        if year1 == year2 and fuzz.ratio(title1, title2) > 80:
            potential_matches.append((title1, title2, year1))

# Create a DataFrame of potential matches
df_matches = pd.DataFrame(potential_matches, columns=['title_df1', 'title_df2', 'year'])

# Merge the DataFrames on the title and year columns
merged_df = pd.merge(df1, df_matches, on=['movie_title', 'movie_year'])
merged_df = pd.merge(merged_df, df2, on=['title', 'year'], suffixes=('_df1', '_df2'))


KeyError: 'movie_title'

In [127]:
print(df1.columns)
print(df2.columns)


Index(['movie_title', 'imdbid', 'id', 'rating', 'movie_year'], dtype='object')
Index(['title', 'rating', 'year', 'users_rating', 'votes', 'metascore',
       'img_url', 'countries', 'languages', 'actors', 'genre', 'tagline',
       'description', 'directors', 'runtime', 'imdb_url'],
      dtype='object')


In [53]:
merged_df = pd.merge(df1, df_matches, left_on='title', right_on='title1')
merged_df = pd.merge(merged_df, df2, left_on='title2', right_on='title', suffixes=('_df1', '_df2'))


In [120]:
merged_df

,title_df1,imdbid,id,rating_df1,year_df1,title1,title2,score,title_df2,rating_df2,...,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,Wrestling,242148.0,9948,0,1892,Wrestling,Wrestling,100,Wrestling,NaN,...,https://m.media-amazon.com/images/M/MV5BMTgxOD...,['USA'],['English'],"['Mark Welling', 'Jeff Conaway', 'Lauren Schne...","['Drama', 'Romance']",Can you ever get it back...,"In the summer between high school and college,...","[""Jeremy O'Keefe""]",91 min,https://www.imdb.com/title/tt0425656/
1,Falling Cat,2049440.0,9662,0,1894,Falling Cat,Falling Apart,83,Falling Apart,NaN,...,https://m.media-amazon.com/images/M/MV5BY2QxMj...,['USA'],['English'],"['Jay Northup', 'Royce Haas', 'Cheryl Allie', ...",['Horror'],NaN,Mark Wilson has been having nightmares of the ...,['Mike Tesh'],NaN,https://www.imdb.com/title/tt0808262/
2,"Great Train Robbery, The",439.0,1319,0,1903,"Great Train Robbery, The",The Great Train Robbery,81,The Great Train Robbery,Approved,...,https://m.media-amazon.com/images/M/MV5BYzAxND...,['USA'],['English'],"['Bob Steele', 'Claire Carleton', 'Milburn Sto...","['Action', 'Crime', 'Drama', 'Thriller']",It's the most amazing adventure of your life.....,Tom Logan is a railroad detective. Tom takes i...,['Joseph Kane'],62 min,https://www.imdb.com/title/tt0033680/
3,A Trip to Mars,1515725.0,5685,0,1910,A Trip to Mars,A Trip to Paris,90,A Trip to Paris,Approved,...,https://m.media-amazon.com/images/M/MV5BYWNmNj...,['USA'],['English'],"['Jed Prouty', 'Shirley Deane', 'Spring Byingt...","['Adventure', 'Comedy']",NaN,The Jones Family goes to Paris to celebrate Fa...,['Malcolm St. Clair'],64 min,https://www.imdb.com/title/tt0030894/
4,Suspense,3424.0,5915,0,1913,Suspense,Suspense,100,Suspense,Passed,...,https://m.media-amazon.com/images/M/MV5BM2E0OD...,['USA'],['English'],"['Belita', 'Barry Sullivan', 'Bonita Granville...","['Drama', 'Film-Noir', 'Romance']",NaN,The proprietor of an ice-skating revue promote...,['Frank Tuttle'],101 min,https://www.imdb.com/title/tt0039000/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,In the Heights,1321510.0,9889,3,2021,In the Heights,In the Night,85,In the Night,Not Rated,...,https://m.media-amazon.com/images/M/MV5BMzA1OT...,"['USA', 'Germany']",['English'],"['Nicole Dionne', 'Eric Dysart', 'Heather Hurs...","['Comedy', 'Music', 'Musical']",NaN,Jane Fisher is a playwright with one night lef...,['Götz Neumann'],86 min,https://www.imdb.com/title/tt2083223/
3173,Luca,12801262.0,9964,1,2021,Luca,Lucas,89,Lucas,PG-13,...,https://m.media-amazon.com/images/M/MV5BYjc3Mj...,['USA'],['English'],"['Corey Haim', 'Kerri Green', 'Charlie Sheen',...","['Comedy', 'Drama', 'Romance']",It's about falling in love...for the first time.,A socially inept fourteen-year-old experiences...,['David Seltzer'],100 min,https://www.imdb.com/title/tt0091445/
3174,"Harder They Fall, The",10696784.0,10123,3,2021,"Harder They Fall, The",Harder They Fall,86,Harder They Fall,R,...,https://m.media-amazon.com/images/M/MV5BMTg2OD...,['USA'],['English'],"['Joshua Lamboy', 'Johnny Vieira', 'Rona D. Le...","['Action', 'Drama', 'Sport']",Courage is not tested until you've been in a r...,20 year old kickboxers from opposite ends of M...,['Lee Cipolla'],NaN,https://www.imdb.com/title/tt0464039/
3175,Eternals,9032400.0,10118,3,2021,Eternals,Eternal,93,Eternal,NaN,...,https://m.media-amazon.com/images/M/MV5BYzRhYz...,['USA'],['English'],"['P.J. Soles', 'Bryan Webb', 'Sarah J. Rutherf...",['Horror'],NaN,NaN,['Derek Rimelspach'],NaN,https://www.imdb.com/title/tt1124034/


In [55]:
# Calculate percentage of missing values for each column
nan_percentage = merged_df.isna().mean() * 100

# Print the percentage of missing values for each column
print(nan_percentage)


title_df1        0.000000
imdbid           0.000000
id               0.000000
rating_df1       0.000000
year_df1         0.000000
title1           0.000000
title2           0.000000
score            0.000000
title_df2        0.000000
rating_df2      21.492220
year_df2         0.000000
users_rating     0.000000
votes            0.000000
metascore       57.223325
img_url          3.379657
countries        0.000000
languages        0.000000
actors           0.000000
genre            0.000000
tagline         27.351217
description      2.576296
directors        0.300106
runtime         14.543608
imdb_url         0.000000
dtype: float64


In [56]:
# Check for duplicate movie titles in the final dataframe
duplicates = merged_df.duplicated(subset='title_df1')

# Count the number of duplicates
num_duplicates = sum(duplicates)

# Print the number of duplicates
print("Number of duplicate movie titles: ", num_duplicates)

Number of duplicate movie titles:  16442


In [58]:
merged_df = merged_df.drop_duplicates(subset='title_df1', keep=False)

# Check the new number of rows in the dataframe
print("Number of rows in the cleaned dataframe: ", len(merged_df))

Number of rows in the cleaned dataframe:  3177


In [59]:
merged_df

,title_df1,imdbid,id,rating_df1,year_df1,title1,title2,score,title_df2,rating_df2,...,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,Wrestling,242148.0,9948,0,1892,Wrestling,Wrestling,100,Wrestling,NaN,...,https://m.media-amazon.com/images/M/MV5BMTgxOD...,['USA'],['English'],"['Mark Welling', 'Jeff Conaway', 'Lauren Schne...","['Drama', 'Romance']",Can you ever get it back...,"In the summer between high school and college,...","[""Jeremy O'Keefe""]",91 min,https://www.imdb.com/title/tt0425656/
1,Falling Cat,2049440.0,9662,0,1894,Falling Cat,Falling Apart,83,Falling Apart,NaN,...,https://m.media-amazon.com/images/M/MV5BY2QxMj...,['USA'],['English'],"['Jay Northup', 'Royce Haas', 'Cheryl Allie', ...",['Horror'],NaN,Mark Wilson has been having nightmares of the ...,['Mike Tesh'],NaN,https://www.imdb.com/title/tt0808262/
493,"Great Train Robbery, The",439.0,1319,0,1903,"Great Train Robbery, The",The Great Train Robbery,81,The Great Train Robbery,Approved,...,https://m.media-amazon.com/images/M/MV5BYzAxND...,['USA'],['English'],"['Bob Steele', 'Claire Carleton', 'Milburn Sto...","['Action', 'Crime', 'Drama', 'Thriller']",It's the most amazing adventure of your life.....,Tom Logan is a railroad detective. Tom takes i...,['Joseph Kane'],62 min,https://www.imdb.com/title/tt0033680/
494,A Trip to Mars,1515725.0,5685,0,1910,A Trip to Mars,A Trip to Paris,90,A Trip to Paris,Approved,...,https://m.media-amazon.com/images/M/MV5BYWNmNj...,['USA'],['English'],"['Jed Prouty', 'Shirley Deane', 'Spring Byingt...","['Adventure', 'Comedy']",NaN,The Jones Family goes to Paris to celebrate Fa...,['Malcolm St. Clair'],64 min,https://www.imdb.com/title/tt0030894/
720,Suspense,3424.0,5915,0,1913,Suspense,Suspense,100,Suspense,Passed,...,https://m.media-amazon.com/images/M/MV5BM2E0OD...,['USA'],['English'],"['Belita', 'Barry Sullivan', 'Bonita Granville...","['Drama', 'Film-Noir', 'Romance']",NaN,The proprietor of an ice-skating revue promote...,['Frank Tuttle'],101 min,https://www.imdb.com/title/tt0039000/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21615,In the Heights,1321510.0,9889,3,2021,In the Heights,In the Night,85,In the Night,Not Rated,...,https://m.media-amazon.com/images/M/MV5BMzA1OT...,"['USA', 'Germany']",['English'],"['Nicole Dionne', 'Eric Dysart', 'Heather Hurs...","['Comedy', 'Music', 'Musical']",NaN,Jane Fisher is a playwright with one night lef...,['Götz Neumann'],86 min,https://www.imdb.com/title/tt2083223/
21621,Luca,12801262.0,9964,1,2021,Luca,Lucas,89,Lucas,PG-13,...,https://m.media-amazon.com/images/M/MV5BYjc3Mj...,['USA'],['English'],"['Corey Haim', 'Kerri Green', 'Charlie Sheen',...","['Comedy', 'Drama', 'Romance']",It's about falling in love...for the first time.,A socially inept fourteen-year-old experiences...,['David Seltzer'],100 min,https://www.imdb.com/title/tt0091445/
21650,"Harder They Fall, The",10696784.0,10123,3,2021,"Harder They Fall, The",Harder They Fall,86,Harder They Fall,R,...,https://m.media-amazon.com/images/M/MV5BMTg2OD...,['USA'],['English'],"['Joshua Lamboy', 'Johnny Vieira', 'Rona D. Le...","['Action', 'Drama', 'Sport']",Courage is not tested until you've been in a r...,20 year old kickboxers from opposite ends of M...,['Lee Cipolla'],NaN,https://www.imdb.com/title/tt0464039/
21655,Eternals,9032400.0,10118,3,2021,Eternals,Eternal,93,Eternal,NaN,...,https://m.media-amazon.com/images/M/MV5BYzRhYz...,['USA'],['English'],"['P.J. Soles', 'Bryan Webb', 'Sarah J. Rutherf...",['Horror'],NaN,NaN,['Derek Rimelspach'],NaN,https://www.imdb.com/title/tt1124034/


In [60]:
# Reset the index of df_matches_dropped
merged_df = merged_df.reset_index(drop=True)


In [61]:
merged_df

,title_df1,imdbid,id,rating_df1,year_df1,title1,title2,score,title_df2,rating_df2,...,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,Wrestling,242148.0,9948,0,1892,Wrestling,Wrestling,100,Wrestling,NaN,...,https://m.media-amazon.com/images/M/MV5BMTgxOD...,['USA'],['English'],"['Mark Welling', 'Jeff Conaway', 'Lauren Schne...","['Drama', 'Romance']",Can you ever get it back...,"In the summer between high school and college,...","[""Jeremy O'Keefe""]",91 min,https://www.imdb.com/title/tt0425656/
1,Falling Cat,2049440.0,9662,0,1894,Falling Cat,Falling Apart,83,Falling Apart,NaN,...,https://m.media-amazon.com/images/M/MV5BY2QxMj...,['USA'],['English'],"['Jay Northup', 'Royce Haas', 'Cheryl Allie', ...",['Horror'],NaN,Mark Wilson has been having nightmares of the ...,['Mike Tesh'],NaN,https://www.imdb.com/title/tt0808262/
2,"Great Train Robbery, The",439.0,1319,0,1903,"Great Train Robbery, The",The Great Train Robbery,81,The Great Train Robbery,Approved,...,https://m.media-amazon.com/images/M/MV5BYzAxND...,['USA'],['English'],"['Bob Steele', 'Claire Carleton', 'Milburn Sto...","['Action', 'Crime', 'Drama', 'Thriller']",It's the most amazing adventure of your life.....,Tom Logan is a railroad detective. Tom takes i...,['Joseph Kane'],62 min,https://www.imdb.com/title/tt0033680/
3,A Trip to Mars,1515725.0,5685,0,1910,A Trip to Mars,A Trip to Paris,90,A Trip to Paris,Approved,...,https://m.media-amazon.com/images/M/MV5BYWNmNj...,['USA'],['English'],"['Jed Prouty', 'Shirley Deane', 'Spring Byingt...","['Adventure', 'Comedy']",NaN,The Jones Family goes to Paris to celebrate Fa...,['Malcolm St. Clair'],64 min,https://www.imdb.com/title/tt0030894/
4,Suspense,3424.0,5915,0,1913,Suspense,Suspense,100,Suspense,Passed,...,https://m.media-amazon.com/images/M/MV5BM2E0OD...,['USA'],['English'],"['Belita', 'Barry Sullivan', 'Bonita Granville...","['Drama', 'Film-Noir', 'Romance']",NaN,The proprietor of an ice-skating revue promote...,['Frank Tuttle'],101 min,https://www.imdb.com/title/tt0039000/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,In the Heights,1321510.0,9889,3,2021,In the Heights,In the Night,85,In the Night,Not Rated,...,https://m.media-amazon.com/images/M/MV5BMzA1OT...,"['USA', 'Germany']",['English'],"['Nicole Dionne', 'Eric Dysart', 'Heather Hurs...","['Comedy', 'Music', 'Musical']",NaN,Jane Fisher is a playwright with one night lef...,['Götz Neumann'],86 min,https://www.imdb.com/title/tt2083223/
3173,Luca,12801262.0,9964,1,2021,Luca,Lucas,89,Lucas,PG-13,...,https://m.media-amazon.com/images/M/MV5BYjc3Mj...,['USA'],['English'],"['Corey Haim', 'Kerri Green', 'Charlie Sheen',...","['Comedy', 'Drama', 'Romance']",It's about falling in love...for the first time.,A socially inept fourteen-year-old experiences...,['David Seltzer'],100 min,https://www.imdb.com/title/tt0091445/
3174,"Harder They Fall, The",10696784.0,10123,3,2021,"Harder They Fall, The",Harder They Fall,86,Harder They Fall,R,...,https://m.media-amazon.com/images/M/MV5BMTg2OD...,['USA'],['English'],"['Joshua Lamboy', 'Johnny Vieira', 'Rona D. Le...","['Action', 'Drama', 'Sport']",Courage is not tested until you've been in a r...,20 year old kickboxers from opposite ends of M...,['Lee Cipolla'],NaN,https://www.imdb.com/title/tt0464039/
3175,Eternals,9032400.0,10118,3,2021,Eternals,Eternal,93,Eternal,NaN,...,https://m.media-amazon.com/images/M/MV5BYzRhYz...,['USA'],['English'],"['P.J. Soles', 'Bryan Webb', 'Sarah J. Rutherf...",['Horror'],NaN,NaN,['Derek Rimelspach'],NaN,https://www.imdb.com/title/tt1124034/


In [62]:
# Calculate percentage of missing values for each column
nan_percentage = merged_df.isna().mean() * 100

# Print the percentage of missing values for each column
print(nan_percentage)

title_df1        0.000000
imdbid           0.000000
id               0.000000
rating_df1       0.000000
year_df1         0.000000
title1           0.000000
title2           0.000000
score            0.000000
title_df2        0.000000
rating_df2       5.949008
year_df2         0.000000
users_rating     0.000000
votes            0.000000
metascore       29.209946
img_url          0.661001
countries        0.000000
languages        0.000000
actors           0.000000
genre            0.000000
tagline         15.140069
description      1.259049
directors        0.125905
runtime          5.823104
imdb_url         0.000000
dtype: float64


In [63]:
merged_df

,title_df1,imdbid,id,rating_df1,year_df1,title1,title2,score,title_df2,rating_df2,...,img_url,countries,languages,actors,genre,tagline,description,directors,runtime,imdb_url
0,Wrestling,242148.0,9948,0,1892,Wrestling,Wrestling,100,Wrestling,NaN,...,https://m.media-amazon.com/images/M/MV5BMTgxOD...,['USA'],['English'],"['Mark Welling', 'Jeff Conaway', 'Lauren Schne...","['Drama', 'Romance']",Can you ever get it back...,"In the summer between high school and college,...","[""Jeremy O'Keefe""]",91 min,https://www.imdb.com/title/tt0425656/
1,Falling Cat,2049440.0,9662,0,1894,Falling Cat,Falling Apart,83,Falling Apart,NaN,...,https://m.media-amazon.com/images/M/MV5BY2QxMj...,['USA'],['English'],"['Jay Northup', 'Royce Haas', 'Cheryl Allie', ...",['Horror'],NaN,Mark Wilson has been having nightmares of the ...,['Mike Tesh'],NaN,https://www.imdb.com/title/tt0808262/
2,"Great Train Robbery, The",439.0,1319,0,1903,"Great Train Robbery, The",The Great Train Robbery,81,The Great Train Robbery,Approved,...,https://m.media-amazon.com/images/M/MV5BYzAxND...,['USA'],['English'],"['Bob Steele', 'Claire Carleton', 'Milburn Sto...","['Action', 'Crime', 'Drama', 'Thriller']",It's the most amazing adventure of your life.....,Tom Logan is a railroad detective. Tom takes i...,['Joseph Kane'],62 min,https://www.imdb.com/title/tt0033680/
3,A Trip to Mars,1515725.0,5685,0,1910,A Trip to Mars,A Trip to Paris,90,A Trip to Paris,Approved,...,https://m.media-amazon.com/images/M/MV5BYWNmNj...,['USA'],['English'],"['Jed Prouty', 'Shirley Deane', 'Spring Byingt...","['Adventure', 'Comedy']",NaN,The Jones Family goes to Paris to celebrate Fa...,['Malcolm St. Clair'],64 min,https://www.imdb.com/title/tt0030894/
4,Suspense,3424.0,5915,0,1913,Suspense,Suspense,100,Suspense,Passed,...,https://m.media-amazon.com/images/M/MV5BM2E0OD...,['USA'],['English'],"['Belita', 'Barry Sullivan', 'Bonita Granville...","['Drama', 'Film-Noir', 'Romance']",NaN,The proprietor of an ice-skating revue promote...,['Frank Tuttle'],101 min,https://www.imdb.com/title/tt0039000/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,In the Heights,1321510.0,9889,3,2021,In the Heights,In the Night,85,In the Night,Not Rated,...,https://m.media-amazon.com/images/M/MV5BMzA1OT...,"['USA', 'Germany']",['English'],"['Nicole Dionne', 'Eric Dysart', 'Heather Hurs...","['Comedy', 'Music', 'Musical']",NaN,Jane Fisher is a playwright with one night lef...,['Götz Neumann'],86 min,https://www.imdb.com/title/tt2083223/
3173,Luca,12801262.0,9964,1,2021,Luca,Lucas,89,Lucas,PG-13,...,https://m.media-amazon.com/images/M/MV5BYjc3Mj...,['USA'],['English'],"['Corey Haim', 'Kerri Green', 'Charlie Sheen',...","['Comedy', 'Drama', 'Romance']",It's about falling in love...for the first time.,A socially inept fourteen-year-old experiences...,['David Seltzer'],100 min,https://www.imdb.com/title/tt0091445/
3174,"Harder They Fall, The",10696784.0,10123,3,2021,"Harder They Fall, The",Harder They Fall,86,Harder They Fall,R,...,https://m.media-amazon.com/images/M/MV5BMTg2OD...,['USA'],['English'],"['Joshua Lamboy', 'Johnny Vieira', 'Rona D. Le...","['Action', 'Drama', 'Sport']",Courage is not tested until you've been in a r...,20 year old kickboxers from opposite ends of M...,['Lee Cipolla'],NaN,https://www.imdb.com/title/tt0464039/
3175,Eternals,9032400.0,10118,3,2021,Eternals,Eternal,93,Eternal,NaN,...,https://m.media-amazon.com/images/M/MV5BYzRhYz...,['USA'],['English'],"['P.J. Soles', 'Bryan Webb', 'Sarah J. Rutherf...",['Horror'],NaN,NaN,['Derek Rimelspach'],NaN,https://www.imdb.com/title/tt1124034/


In [66]:
for index, row in merged_df.iterrows():
    if row['title_df1'] != row['title1']:
        print(index, row['title_df1'], row['title1'])


In [70]:
for index, row in merged_df.iterrows():
    if row['title1'] != row['title2']:
        print(index, row['title1'], row['title2'])


1 Falling Cat Falling Apart
2 Great Train Robbery, The The Great Train Robbery
3 A Trip to Mars A Trip to Paris
7 Intolerance: Love&#39;s Struggle Throughout the Ages Intolerance: Love's Struggle Throughout the Ages
8 A Romance of the Redwoods Romance of the Redwoods
10 Cabinet of Dr. Caligari, The The Cabinet of Dr. Caligari
11 Within Our Gates Huns Within Our Gates
19 So&#39;s Your Old Man So's Your Old Man
20 Kid Brother, The Kid Brother
23 Power of the Press, The Power of the Press
25 Docks of New York, The Docks of New York
26 Man with a Movie Camera Monster with a Movie Camera
27 Broadway Melody The Broadway Melody
35 Public Enemy, The Public Enemy #2
41 Vampyr Vampyre
58 A Night At The Opera A Night at the Opera
59 Bride of Frankenstein, The Bride of Frankenstein
62 Charge of the Light Brigade, The The Charge of the Light Brigade
66 Dracula&#39;s Daughter Dracula's Daughter
70 Knight Without Armor A Night Without Armor
77 Dawn Patrol, The Dawn Patrol
80 Adventures of Robin Hood,

In [90]:
# Boolean indexing to filter rows where title_df2 and title2 are not the same, and score < 83
diff_title_rows = merged_df[merged_df['title_df1'] != merged_df['title2']]
diff_title_score_rows = diff_title_rows[diff_title_rows['score'] < 82]

# Display the resulting rows
print(diff_title_score_rows)


                                      title_df1     imdbid     id  rating_df1  \
2                      Great Train Robbery, The      439.0   1319           0   
20                             Kid Brother, The    18051.0   1964           0   
35                            Public Enemy, The    22286.0   5934           3   
77                             Dawn Patrol, The    30044.0   1238           0   
121                    Major and the Minor, The    35019.0    681           3   
163                    Ghost and Mrs. Muir, The    39420.0   3047           3   
199                          Tarzan&#39;s Peril    43695.0   9703           1   
228                            Journey to Italy    46511.0   3954           3   
234                    Night of the Hunter, The    48424.0    878           3   
262                    Wild Women of Wongo, The    52394.0   2585           3   
275                         Come Dance with Me!    53428.0   3442           3   
335                    Troub

In [102]:
diff_title_score_rows[['title_df1','title_df2','year_df1', 'year_df2','imdb_url']]

,title_df1,title_df2,year_df1,year_df2,imdb_url
2,"Great Train Robbery, The",The Great Train Robbery,1903,1941.0,https://www.imdb.com/title/tt0033680/
20,"Kid Brother, The",Kid Brother,1927,2017.0,https://www.imdb.com/title/tt7069030/
35,"Public Enemy, The",Public Enemy #2,1931,1991.0,https://www.imdb.com/title/tt0107896/
77,"Dawn Patrol, The",Dawn Patrol,1938,2014.0,https://www.imdb.com/title/tt2073661/
121,"Major and the Minor, The",The Major and the Minor,1942,1942.0,https://www.imdb.com/title/tt0035019/
163,"Ghost and Mrs. Muir, The",The Ghost and Mrs. Muir,1947,1947.0,https://www.imdb.com/title/tt0039420/
199,Tarzan&#39;s Peril,Tarzan's Peril,1951,1951.0,https://www.imdb.com/title/tt0043695/
228,Journey to Italy,Journey to Lasta,1954,2004.0,https://www.imdb.com/title/tt0436455/
234,"Night of the Hunter, The",The Night of the Hunter,1955,1955.0,https://www.imdb.com/title/tt0048424/
262,"Wild Women of Wongo, The",The Wild Women of Wongo,1958,1958.0,https://www.imdb.com/title/tt0052394/


In [ ]:
rows_to_drop = [35, 77, 228, 275, 368, 389, 439, 638]


In [96]:
merged_df[['year_df1', 'year_df2']]

,year_df1,year_df2
0,1892,2008.0
1,1894,2005.0
2,1903,1941.0
3,1910,1938.0
4,1913,1946.0
...,...,...
3172,2021,2012.0
3173,2021,1986.0
3174,2021,2005.0
3175,2021,2012.0


In [ ]:
merged_df['year_df2'] = merged_df['year_df2'].astype('int64')


In [93]:

# Boolean indexing to filter rows where title_df2 and title2 are not the same, and score < 83
diff_year_rows = merged_df[merged_df['year_df1'] != merged_df['year_df2']]

# Display the resulting rows
print(diff_year_rows)

                     title_df1      imdbid     id  rating_df1  year_df1  \
0                    Wrestling    242148.0   9948           0      1892   
1                  Falling Cat   2049440.0   9662           0      1894   
2     Great Train Robbery, The       439.0   1319           0      1903   
3               A Trip to Mars   1515725.0   5685           0      1910   
4                     Suspense      3424.0   5915           0      1913   
...                        ...         ...    ...         ...       ...   
3172            In the Heights   1321510.0   9889           3      2021   
3173                      Luca  12801262.0   9964           1      2021   
3174     Harder They Fall, The  10696784.0  10123           3      2021   
3175                  Eternals   9032400.0  10118           3      2021   
3176                   Respect   2452150.0  10106           3      2021   

                        title1                   title2  score  \
0                    Wrestling   

In [121]:
diff_year_rows[['title_df1','title_df2','year_df1', 'year_df2','imdb_url']]

,title_df1,title_df2,year_df1,year_df2,imdb_url
0,Wrestling,Wrestling,1892,2008.0,https://www.imdb.com/title/tt0425656/
1,Falling Cat,Falling Apart,1894,2005.0,https://www.imdb.com/title/tt0808262/
2,"Great Train Robbery, The",The Great Train Robbery,1903,1941.0,https://www.imdb.com/title/tt0033680/
3,A Trip to Mars,A Trip to Paris,1910,1938.0,https://www.imdb.com/title/tt0030894/
4,Suspense,Suspense,1913,1946.0,https://www.imdb.com/title/tt0039000/
...,...,...,...,...,...
3172,In the Heights,In the Night,2021,2012.0,https://www.imdb.com/title/tt2083223/
3173,Luca,Lucas,2021,1986.0,https://www.imdb.com/title/tt0091445/
3174,"Harder They Fall, The",Harder They Fall,2021,2005.0,https://www.imdb.com/title/tt0464039/
3175,Eternals,Eternal,2021,2012.0,https://www.imdb.com/title/tt1124034/
